# Fake news prediction

We are going to use Supervised learning method i.e. labelled data and will preprocess the data (Convert the data to numerical) so that machine can understand it and then will split the data into training and testing data. We are going to use Logistic regression model as it is binary classification model.

## Importing neccessary libraries

In [84]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
nltk.download('punkt')
import re
import nltk.corpus
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords #To remove stopwords
from nltk.stem.porter import PorterStemmer #To perform the Stemming function
from sklearn.feature_extraction.text import TfidfVectorizer #to create TF IDF
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\goelp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\goelp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Printing the stopwords in english

In [3]:
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Pre-processing

Loading the dataset into pandas dataframe

In [4]:
news_dataset = pd.read_csv("C:/Users/goelp/Desktop/Python Projects/Fake News Prediction/train.csv")
news_dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head(7)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1


In [7]:
news_dataset.title.head(7)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
5    Jackie Mason: Hollywood Would Love Trump if He...
6    Life: Life Of Luxury: Elton John’s 6 Favorite ...
Name: title, dtype: object

In [9]:
news_dataset.text.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    Ever get the feeling your life circles the rou...
2    Why the Truth Might Get You Fired October 29, ...
3    Videos 15 Civilians Killed In Single US Airstr...
4    Print \nAn Iranian woman has been sentenced to...
Name: text, dtype: object

In [10]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

As we can see, there are 558 null values in title and approx 2000 missing values in author so we are going to drop the missing values as total dataset contains 20,000 dataset.
We can drop the values as our dataset is large.

In [11]:
news_dataset = news_dataset.fillna("")

In [12]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [13]:
news_dataset["content"] = news_dataset["author"]+" "+news_dataset["title"]
print(news_dataset)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [14]:
X = news_dataset.drop(columns = "label", axis=1)
Y = news_dataset["label"]

In [15]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
    #Using regex to detect alphabets
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    #lowercasing the characters
    stemmed_content = stemmed_content.lower()
    #splitting the lowercased content
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [20]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [21]:
news_dataset['content']

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

Creating features and labels

In [22]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [23]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [24]:
print(Y)

[1 0 1 ... 0 1 1]


In [25]:
Y.shape

(20800,)

In [26]:
X.shape

(20800,)

Using TF-IDF Vectoriser to convert text data into numerical data

In [27]:
vectoriser = TfidfVectorizer()

In [28]:
vectoriser.fit(X)

TfidfVectorizer()

In [29]:
X = vectoriser.transform(X)

In [30]:
print(X)

  (0, 15545)	0.28485063562728646
  (0, 13342)	0.2565896679337957
  (0, 8818)	0.3635963806326075
  (0, 8542)	0.29212514087043684
  (0, 7608)	0.24785219520671603
  (0, 6928)	0.21874169089359144
  (0, 4916)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16646)	0.3025156488372128
  (1, 6742)	0.19152496072048605
  (1, 5439)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15470)	0.41544962664721613
  (2, 9528)	0.49351492943649944
  (2, 5901)	0.3474613386728292
  (2, 5325)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12992)	0.2483705036831893
  (20797, 12222)	0.27276402145717243
  (20797, 12017)	0.24790022252744132
  (20797, 10212)	0.0804189541935242
  (20797, 9496)	0.17463635692029988
  (20797, 9427)	0.29394

Splitting the dataset into Training and testing data

In [69]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=7)

In [70]:
X_train.shape

(16640, 16974)

In [71]:
X_test.shape

(4160, 16974)

In [72]:
Y_train.shape

(16640,)

In [73]:
Y_test.shape

(4160,)

So now that we have created training and testing data, We are going to train our model based on the training data

In [74]:
model = LogisticRegression()

In [75]:
model.fit(X_train, Y_train)

LogisticRegression()

In [76]:
preds = model.predict(X_train)

In [77]:
preds.shape

(16640,)

In [79]:
accuracy_score(preds, Y_train)

0.9870793269230769

Checking the model's accuracy on Test data

In [80]:
preds_test = model.predict(X_test)

In [81]:
accuracy_score(Y_test, preds_test)

0.9776442307692308

Our model is performing really well and has not overtrained on the dataset as we are getting equal accuracy for both training and testing data

Making a predictive system to use the above model for prediction purpose

In [88]:
newscheck = input("Enter your news article here: ")

# Tokenize and stem the input text
newscheck_tokens = nltk.word_tokenize(newscheck)
newscheck_stemmed = [port_stem.stem(word) for word in newscheck_tokens]
newscheck_processed = ' '.join(newscheck_stemmed)

# Transform the user input into a feature vector
user_vector = vectoriser.transform([newscheck_processed])

# Assuming you have a trained model called 'model'
user_prediction = model.predict(user_vector)

if user_prediction[0] == 0:
    print('The news is real')
else:
    print('The news is fake')

Enter your news article here: Dr. Fabio Mythic, the lead scientist on the expedition, explained, "These penguins don't just make random noises. They actually seem to be communicating in a structured language, reminiscent of some human languages." The research team has tentatively named the species 'Vocalus Penguinus.'  Remarkably, these penguins have been observed forming circles, apparently 'discussing' fishing spots and predator alerts. Some even seem to 'sing' in chorus, producing melodies that, according to Dr. Mythic, "are oddly reminiscent of the songs from the 80s pop band, ABBA."  Critics and skeptics worldwide await peer-reviewed studies. Dr. Real McScientist from the University of Credibility expressed his skepticism, "While the findings are intriguing, we need more tangible evidence. It's essential to approach such claims with a hefty dose of skepticism."
The news is fake
